In [4]:
!pip install pyspark install-jdk findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=92a3ef9c52b1b1d8dcd267f89cdffcfe8bcc9134a44ba66af1ed9c38cefe19fb
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [5]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand, expr, floor
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
import random
from datetime import datetime, timedelta

In [6]:
# Создание SparkSession
spark = SparkSession.builder.master("local").appName("Synthetic Data Generation").getOrCreate()

In [7]:
# Параметры генерации данных
num_rows = 1000  # Количество строк (можно изменить по необходимости)
products_list = ["Laptop", "Smartphone", "Headphones", "Tablet", "Smartwatch"]  # Список возможных товаров

In [8]:
# Функция для генерации случайной даты в пределах последнего года
def random_date():
    start_date = datetime.now() - timedelta(days=365)
    random_days = random.randint(0, 365)
    return (start_date + timedelta(days=random_days)).date()

In [9]:
# Генерация синтетических данных
data = []
for _ in range(num_rows):
    date = random_date()
    user_id = random.randint(1, 100)  # Случайный идентификатор пользователя
    product = random.choice(products_list)
    quantity = random.randint(1, 5)  # Случайное количество
    price = round(random.uniform(10, 500), 2)  # Случайная цена в пределах от 10 до 500
    data.append((date, user_id, product, quantity, price))

In [10]:
# Определение схемы DataFrame
schema = StructType([
    StructField("Date", DateType(), True),
    StructField("UserID", IntegerType(), True),
    StructField("Product", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Price", DoubleType(), True)
])

In [11]:
# Создание DataFrame
orders_df = spark.createDataFrame(data, schema)
orders_df

DataFrame[Date: date, UserID: int, Product: string, Quantity: int, Price: double]

In [12]:
# Сохранение DataFrame в формате CSV
output_path = "orders_data.csv"
orders_df.write.csv(output_path, header=True, mode="overwrite")

In [13]:
# Завершение сессии Spark
spark.stop()